In [3]:
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Taylor\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import requests
import json

def fetch_all_users():
    # Configuração inicial
    url = "https://my.timetrade.com/AccountManager/user/getUserList"
    headers = {
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7",
        "cache-control": "no-cache",
        "origin": "https://my.timetrade.com",
        "pragma": "no-cache",
        "priority": "u=1, i",
        "referer": "https://my.timetrade.com/AccountManager/accountManager/index/user?buySubscription=0&adminEmail=null",
        "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "x-requested-with": "XMLHttpRequest"
    }
    cookies = {
        "JSESSIONID": "2C7E27FB1E9639AB8ABE51A97886EA36",
        "xSession": '{"email":"Megan.Halverson@thomsonreuters.com","hash":"nYn7Crpw9pvPAc4B8VUdLQ==","token":null}',
        "psCookie": '{"ident":"nYn7Crpw9pvPAc4B8VUdLQ==","email":"megan.halverson@thomsonreuters.com","rememberMe":true}',
        "AWSALB": "XybWW4L2vChW38824EVmswQbO6BEkJednT1UMN+ZsX7n/Q5ojElD92AUIjXu/LBQMzyYF1UzAfIeKuzyQNtu3UOXIcfXackcRoPejvi1KAJCmJx2S7bvEs8/axzT",
        "AWSALBCORS": "XybWW4L2vChW38824EVmswQbO6BEkJednT1UMN+ZsX7n/Q5ojElD92AUIjXu/LBQMzyYF1UzAfIeKuzyQNtu3UOXIcfXackcRoPejvi1KAJCmJx2S7bvEs8/axzT"
    }

    offset = 0
    max_items = 15
    all_users = []

    while True:
        # Configurar os parâmetros para paginação
        params = {
            "offset": offset,
            "max": max_items,
            "sort": "statusid",
            "order": "asc"
        }

        # Fazer a requisição POST
        response = requests.post(url, headers=headers, cookies=cookies, data=params)

        if response.status_code != 200:
            print(f"Erro na requisição: {response.status_code}")
            break

        # Processar os dados
        try:
            data = response.json()  # Tentar interpretar o JSON da resposta
            users = data.get("items", [])  # Assumindo que os usuários estão no campo "items"

            if not users:
                print("Nenhum usuário encontrado ou fim da lista.")
                break

            all_users.extend(users)
            offset += max_items  # Atualizar o offset para próxima página

            # Verificar se todos os usuários foram coletados
            if len(users) < max_items:
                print("Última página alcançada.")
                break
        except json.JSONDecodeError:
            print("Erro ao decodificar a resposta como JSON.")
            break

    return all_users

if __name__ == "__main__":
    users = fetch_all_users()
    print(f"Total de usuários coletados: {len(users)}")
    for user in users:
        print(user)


Erro ao decodificar a resposta como JSON.
Total de usuários coletados: 0


In [8]:
import requests
from bs4 import BeautifulSoup
import json

# URL base
base_url = "https://my.timetrade.com/AccountManager/user/getUserList"
headers = {
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7",
        "cache-control": "no-cache",
        "origin": "https://my.timetrade.com",
        "pragma": "no-cache",
        "priority": "u=1, i",
        "referer": "https://my.timetrade.com/AccountManager/accountManager/index/user?buySubscription=0&adminEmail=null",
        "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "x-requested-with": "XMLHttpRequest",
        "JSESSIONID": "2C7E27FB1E9639AB8ABE51A97886EA36",
        "xSession": '{"email":"Megan.Halverson@thomsonreuters.com","hash":"nYn7Crpw9pvPAc4B8VUdLQ==","token":null}',
        "psCookie": '{"ident":"nYn7Crpw9pvPAc4B8VUdLQ==","email":"megan.halverson@thomsonreuters.com","rememberMe":true}',
        "AWSALB": "XybWW4L2vChW38824EVmswQbO6BEkJednT1UMN+ZsX7n/Q5ojElD92AUIjXu/LBQMzyYF1UzAfIeKuzyQNtu3UOXIcfXackcRoPejvi1KAJCmJx2S7bvEs8/axzT",
        "AWSALBCORS": "XybWW4L2vChW38824EVmswQbO6BEkJednT1UMN+ZsX7n/Q5ojElD92AUIjXu/LBQMzyYF1UzAfIeKuzyQNtu3UOXIcfXackcRoPejvi1KAJCmJx2S7bvEs8/axzT"
    }


# Número máximo de registros por página e total de registros
max_records = 15
total_records = 282  # Ajuste caso a quantidade total mude

# Lista para armazenar os dados de todos os usuários
all_users = []

# Função para extrair dados de cada página
def fetch_users(offset):
    payload = {
        "offset": offset,
        "max": max_records,
        "sort": "statusid",
        "order": "asc"
    }

    response = requests.post(base_url, headers=headers, data=payload)
    response.raise_for_status()  # Garante que o código HTTP é 200
    return response.text

# Loop para percorrer todas as páginas
for offset in range(0, total_records, max_records):
    print(f"Fetching users from offset: {offset}")
    html_response = fetch_users(offset)

    # Processa o HTML retornado para extrair o script JSON
    soup = BeautifulSoup(html_response, "html.parser")
    script_tag = soup.find("script", text=lambda t: "var userData" in t)
    
    if script_tag:
        # Extrai o JSON de dados do script
        script_content = script_tag.string
        start_index = script_content.find("{")
        end_index = script_content.rfind("}") + 1
        user_data_json = script_content[start_index:end_index]
        user_data = json.loads(user_data_json)
        
        # Adiciona os usuários da página à lista
        all_users.extend(user_data.get("items", []))
    else:
        print(f"No user data found on offset: {offset}")

# Salva todos os usuários em um arquivo JSON
with open("all_users.json", "w", encoding="utf-8") as f:
    json.dump(all_users, f, indent=4, ensure_ascii=False)

print(f"Fetched a total of {len(all_users)} users.")


Fetching users from offset: 0


C:\Users\Taylor\AppData\Local\Temp\ipykernel_1600\2719886927.py:58: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tag = soup.find("script", text=lambda t: "var userData" in t)


No user data found on offset: 0
Fetching users from offset: 15
No user data found on offset: 15
Fetching users from offset: 30
No user data found on offset: 30
Fetching users from offset: 45
No user data found on offset: 45
Fetching users from offset: 60
No user data found on offset: 60
Fetching users from offset: 75
No user data found on offset: 75
Fetching users from offset: 90
No user data found on offset: 90
Fetching users from offset: 105
No user data found on offset: 105
Fetching users from offset: 120
No user data found on offset: 120
Fetching users from offset: 135
No user data found on offset: 135
Fetching users from offset: 150
No user data found on offset: 150
Fetching users from offset: 165
No user data found on offset: 165
Fetching users from offset: 180
No user data found on offset: 180
Fetching users from offset: 195
No user data found on offset: 195
Fetching users from offset: 210
No user data found on offset: 210
Fetching users from offset: 225
No user data found on of

In [12]:
import requests

# URL alvo
url = "https://my.timetrade.com/AccountManager/user/getUserList?offset=0&max=15&sort=statusid&order=asc"

try:
    # Realizando a requisição
    response = requests.get(url)
    
    # Verificando se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Capturando o conteúdo HTML bruto
        html_content = response.text
        
        # Salvando o conteúdo em um arquivo para análise posterior
        with open("response.html", "w", encoding="utf-8") as file:
            file.write(html_content)
        
        print("Conteúdo salvo em 'response.html'")
    else:
        print(f"Falha ao acessar a URL. Código de status: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"Ocorreu um erro na requisição: {e}")


Conteúdo salvo em 'response.html'
